In [56]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False

In [57]:
df = pd.read_csv('dataset/processed_english_twitter_data.csv', skipinitialspace=True, engine="python")
df.head()

,Lang,Text,Author,Service
0,en,tata crucible b bajaj allianz general insuranc...,MrD_Shukla,insurance
1,en,card made love attention detail every card spe...,dreamsbym_,card
2,en,rt coolfundoo man top stock holding like secre...,Coolfundoo,investment
3,en,rt quicktake south korea unique rental system ...,CaudiaAngelica,savings
4,en,amit hi please avoid sharing sensitive informa...,Bajaj_Finance,emi


In [58]:
classes = list(df.Service.unique())
hm = {}
for i in range(len(classes)):
    hm[classes[i]] = i

df['label'] = 0
for i in range(len(df)):
    df['label'][i] = hm[df['Service'][i]]
    
df.head()

C:\Users\ayush\AppData\Local\Temp\ipykernel_7800\3414255578.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][i] = hm[df['Service'][i]]
C:\Users\ayush\AppData\Local\Temp\ipykernel_7800\3414255578.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][i] = hm[df['Service'][i]]
C:\Users\ayush\AppData\Local\Temp\ipykernel_7800\3414255578.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][i] = hm[df['Ser

,Lang,Text,Author,Service,label
0,en,tata crucible b bajaj allianz general insuranc...,MrD_Shukla,insurance,0
1,en,card made love attention detail every card spe...,dreamsbym_,card,1
2,en,rt coolfundoo man top stock holding like secre...,Coolfundoo,investment,2
3,en,rt quicktake south korea unique rental system ...,CaudiaAngelica,savings,3
4,en,amit hi please avoid sharing sensitive informa...,Bajaj_Finance,emi,4


In [59]:
df.groupby(by='label').count()

,Lang,Text,Author,Service
label,,,,
0,82,82,82,82
1,136,136,136,136
2,149,149,149,149
3,106,106,106,106
4,92,92,92,92
5,153,153,153,153


In [60]:
sentences, labels = list(df.Text), list(df.label)

In [61]:
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [62]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  amit hi please avoid sharing sensitive information may misused fraudsters result financial loss personal detail like mobile number id address proof emi card number etc social medium platform please share via dm
Into a sequence of int: [925, 83, 20, 140, 293, 255, 188, 161, 294, 295, 162, 23, 231, 75, 107, 24, 163, 48, 205, 232, 206, 9, 6, 48, 115, 233, 256, 126, 20, 93, 141, 207]
Into a padded sequence: [925  83  20 140 293 255 188 161 294 295 162  23 231  75 107  24 163  48
 205 232 206   9   6  48 115 233 256 126  20  93 141 207   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0]


In [63]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
rt 2
bajaj 3
http 4
co 5
card 6
loan 7
investment 8
emi 9
finserv 10
3541


In [81]:
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Input, Embedding, Dense, SpatialDropout2D, Dropout
from tensorflow.keras.layers import concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model

def define_model(kernel_size=3, activation='relu', input_dim=None, output_dim=300, max_length=None):
        
        inp = Input(shape=(max_length,6))
        x = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(inp)
        x = SpatialDropout2D(0.1)(x)

        x = TCN(nb_filters=128, dilations=[1, 2, 4], return_sequences=True,
                activation=activation, name='tcn1')(x)
        x = TCN(nb_filters=64, dilations=[1, 2, 4, 8], return_sequences=True,
                activation=activation, name='tcn2')(x)

        avg_pool = GlobalAveragePooling1D()(x)  
        max_pool = GlobalMaxPooling1D()(x)

        conc = concatenate([avg_pool, max_pool])
        conc = Dense(16, activation="tanh")(conc)
        conc = Dropout(0.1)(conc)
        outp = Dense(1, activation="softmax")(conc)

        model = Model(inputs=inp, outputs=outp)
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model


In [82]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_22 (InputLayer)       [(None, 100)]                0         []                            
                                                                                                  
 embedding_21 (Embedding)    (None, 100, 300)             300000    ['input_22[0][0]']            
                                                                                                  
 spatial_dropout1d_14 (Spat  (None, 100, 300)             0         ['embedding_21[0][0]']        
 ialDropout1D)                                                                                    
                                                                                                  
 tcn1 (TCN)                  (None, 100, 128)             400256    ['spatial_dropout1d_14[

In [66]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                            patience=10, verbose=2, 
                                            mode='auto', restore_best_weights=True)

In [55]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['tanh']
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(n_splits=10, shuffle=True)

# Separate the sentences and the labels
sentences, labels = list(df.Text), list(df.label)

exp = 0

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1

            # Define the input shape
            model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                    callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

-------------------------------------------
Training 1: tanh activation, 1 kernel size.
-------------------------------------------
Epoch 1/100


d:\Projects\Web scraping and Automation\env\lib\site-packages\tensorflow\python\util\dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


13/13 [==============================] - 5s 162ms/step - loss: 0.0000e+00 - accuracy: 0.1950 - val_loss: 0.0000e+00 - val_accuracy: 0.1389
Epoch 2/100
13/13 [==============================] - 2s 122ms/step - loss: 0.0000e+00 - accuracy: 0.1950 - val_loss: 0.0000e+00 - val_accuracy: 0.1389
Epoch 3/100
13/13 [==============================] - 2s 121ms/step - loss: 0.0000e+00 - accuracy: 0.1950 - val_loss: 0.0000e+00 - val_accuracy: 0.1389
Epoch 4/100
13/13 [==============================] - 2s 122ms/step - loss: 0.0000e+00 - accuracy: 0.1950 - val_loss: 0.0000e+00 - val_accuracy: 0.1389
Epoch 5/100
13/13 [==============================] - 2s 124ms/step - loss: 0.0000e+00 - accuracy: 0.1950 - val_loss: 0.0000e+00 - val_accuracy: 0.1389
Epoch 6/100
13/13 [==============================] - 2s 124ms/step - loss: 0.0000e+00 - accuracy: 0.1950 - val_loss: 0.0000e+00 - val_accuracy: 0.1389
Epoch 7/100
13/13 [==============================] - 2s 124ms/step - loss: 0.0000e+00 - accuracy: 0.1950 -

KeyboardInterrupt: 